In [1]:
# import internal .py modules
import file_path_management as fpath
import public_library as plib

2023-07-31 22:18:39 Didis-MacBook-Pro.local metapub.config[791] WARNING NCBI_API_KEY was not set.


In [2]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re

In [3]:
def preprocess_google_shcolar(source_path, output_path, columns):
    print("Starting merging search results from Google Scholar...")

    df = pd.read_csv(source_path, header=None, sep=',')
    df.columns = ["title", "url", "full_text_url", "full_text_type", "full_text_source"]

    for ind in df.index:
        # if a row doesn't have url or title, then skip and drop this row
        if (df["title"][ind] != df["title"][ind]) or (df["url"][ind] != df["url"][ind]):
            continue

        # get DOI
        doi = np.nan

        # get pmid from DOI
        pmid = plib.doi2pmid(doi)

        # get pmcid from pmid
        pmcid = np.nan

        # get full_text_url, full_text_source
        if str(df["full_text_type"][ind]) == "[HTML]":
            full_text_url, = plib.get_final_redirected_url(str(df["full_text_url"][ind]).strip()).strip()
            full_text_source = full_text_url.split("://")[1].strip().split("/")[0].strip()
        elif str(df["full_text_type"][ind]) == "not found" | "[PDF]" | "UB":
            if str(df["url"][ind]) != "not found":
                full_text_url, = plib.get_final_redirected_url(str(df["url"][ind]).strip()).strip()
                full_text_source = full_text_url.split("://")[1].strip().split("/")[0].strip()
            else:
                full_text_url = np.nan
                full_text_source = np.nan
        else:
            raise Exception("Found a full_text_type not expected!")
        
        # get pdf_url, pdf_source
        if str(df["full_text_type"][ind]) == "[PDF]":
            pdf_url = str(df["full_text_url"][ind])
            pdf_source = pdf_url.split("://")[1].strip().split("/")[0].strip()
        else:
            pdf_url = np.nan
            pdf_source = np.nan
        
        # columns = ["DOI", "PMID", "PMCID", "Title", "full_text_url", "full_text_source", "pdf_url", "pdf_source"]
        row = {
            "DOI": [doi],
            "PMID": [pmid],
            "PMCID": [pmcid],
            "Title": [str(df["title"][ind]).strip()],
            "full_text_url": [full_text_url],
            "full_text_source": [full_text_source],
            "pdf_url": [pdf_url],
            "pdf_source": [pdf_source]
        }
        # print(row)

        if not plib.add_row_to_csv(output_path, row, columns):
            print("Error detected when adding a row to csv!")
        
        print(ind)
# --------------------start of test code--------------------
# source_path = fpath.poten_litera_gs
# output_path = fpath.poten_litera_gs_processed
# plib.clear_file(output_path)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# df = pd.read_csv(source_path, header=None, sep=',')
# df.columns = ["title", "url", "full_text_url", "full_text_type", "full_text_source"]
# print(df.head(3))
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# full_text_type = set(df['full_text_type'].tolist())
# print(full_text_type)
# # {'[HTML]', nan, '[PDF]', 'UB'}
# full_text_source = set(df['full_text_source'].tolist())
# print(full_text_source)
# # {'RWTH-Link', 'ijpp.com', 'nih.gov', 'sciencedirect.com', 'koreamed.org', 'biomedcentral.com', 
# # 'ekja.org', 'harvard.edu', 'bu.edu', 'europepmc.org', 'lww.com', 'ieee.org', 'oup.com', 'mcgill.ca', 
# # 'uottawa.ca', 'plos.org', 'ahajournals.org', 'cell.com', 'sonar.ch', 'core.ac.uk', 'mdpi.com', 
# # nan, 'wiley.com', 'nature.com', 'uzh.ch', 'elifesciences.org', 'nyu.edu', 'karger.com', 
# # 'academia.edu', 'psychiatryonline.org', 'tandfonline.com', 'jordanbpeterson.com', 'duke.edu', 
# # 'science.org', 'pnas.org', 'umich.edu', 'springer.com', 'unav.edu', 'jst.go.jp', 'bmj.com', 
# # 'researchgate.net', 'eneuro.org', 'annualreviews.org', 'northwestern.edu', 'rero.ch', 
# # 'zsp.com.pk', 'epfl.ch', 'jneurosci.org', 'mirasmart.com', 'sagepub.com', 'ahuman.org', 
# # 'psu.edu', 'frontiersin.org', 'scholarpedia.org', 'jpn.ca', 'udc.es', 'biorxiv.org', 'physiology.org', 
# # 'royalsocietypublishing.org', 'escholarship.org', 'wustl.edu', 'ucdavis.edu', 'asahq.org'}
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# ["title", "url", "full_text_url", "full_text_type", "full_text_source"]
# print(df["title"].isnull().any().any())
# print(df["url"].isnull().any().any())
# print(df["full_text_url"].isnull().any().any())
# print(df["full_text_type"].isnull().any().any())
# print(df["full_text_source"].isnull().any().any())
# # # True, True, True, True, True
# # # title, url, full_text_url, full_text_type, full_text_source contain np.nan
# # # we need to fill in what are missing
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# preprocess_google_shcolar(source_path, output_path, columns)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# df = pd.read_csv(output_path, header=None, sep=',')
# print(df.head(3))
# ---------------------end of test code---------------------